In [10]:
import asyncio
import platform
import logging
%autoawait asyncio
import bleak
import importlib
loop = asyncio.get_event_loop()

In [11]:
async def discover():
    devices = await bleak.discover()
    for d in devices:
        print(d)

In [12]:
loop.create_task(discover())

<Task pending coro=<discover() running at <ipython-input-11-71c846b72948>:1> created at <ipython-input-12-1a4ce9146d9a>:1>

In [13]:
address = "7137C3CC-8B3F-4621-A698-118852E4705F"

In [14]:
async def print_services(mac_addr: str, loop: asyncio.AbstractEventLoop):
    async with bleak.BleakClient(mac_addr, loop=loop) as client:
        svcs = await client.get_services()
        print("Services:", svcs)

loop.create_task(print_services(address, loop))

<Task pending coro=<print_services() running at <ipython-input-14-ac79d5be4653>:1> created at <ipython-input-14-ac79d5be4653>:6>

In [15]:
async def run(address, loop, debug=False):
    log = logging.getLogger(__name__)
    if debug:
        import sys

        loop.set_debug(True)
        log.setLevel(logging.DEBUG)
        h = logging.StreamHandler(sys.stdout)
        h.setLevel(logging.DEBUG)
        log.addHandler(h)

    async with bleak.BleakClient(address, loop=loop) as client:
        x = await client.is_connected()
        log.info("Connected: {0}".format(x))

        for service in client.services:
            log.info("[Service] {0}: {1}".format(service.uuid, service.description))
            for char in service.characteristics:
                if "read" in char.properties:
                    try:
                        value = bytes(await client.read_gatt_char(char.uuid))
                    except Exception as e:
                        value = str(e).encode()
                else:
                    value = None
                log.info(
                    "\t[Characteristic] {0}: ({1}) | Name: {2}, Value: {3} ".format(
                        char.uuid, ",".join(char.properties), char.description, value
                    )
                )
                for descriptor in char.descriptors:
                    value = await client.read_gatt_descriptor(descriptor.handle)
                    log.info(
                        "\t\t[Descriptor] {0}: (Handle: {1}) | Value: {2} ".format(
                            descriptor.uuid, descriptor.handle, bytes(value)
                        )
                    )

importlib.reload(logging)
loop = asyncio.get_event_loop()
loop.create_task(run(address, loop, True))

<Task pending coro=<run() running at <ipython-input-15-076322b0edbe>:1> created at <ipython-input-15-076322b0edbe>:41>

In [16]:
characteristic = "6A806050-B5A3-F393-E0A9-E50E24DCCA9E"

In [17]:
def notification_handler(sender, data):
    """Simple notification handler which prints the data received."""
    print("{0}: {1}".format(sender, data))


async def run(address, loop, debug=False):
    log = logging.getLogger(__name__)
    if debug:
        import sys

        loop.set_debug(True)
        log.setLevel(logging.DEBUG)
        h = logging.StreamHandler(sys.stdout)
        h.setLevel(logging.DEBUG)
        log.addHandler(h)

    async with bleak.BleakClient(address, loop=loop) as client:
        x = await client.is_connected()
        logger.info("Connected: {0}".format(x))
#         await client.start_notify(characteristic, notification_handler)
#         await asyncio.sleep(5.0, loop=loop)
#         await client.stop_notify(characteristic)

importlib.reload(logging)
loop = asyncio.get_event_loop()
loop.create_task(run(address, loop, True))

<Task pending coro=<run() running at <ipython-input-17-e8866e4c7c11>:6> created at <ipython-input-17-e8866e4c7c11>:26>

2B07654A-26AB-4F7B-941C-25D9E1A2D8C4: B4E5533C40B830AEE9
AD84BD0C-155F-4798-9BBC-84CDA9D767BF: Microsoft (010920020e83618007f30d9c19ad939ecab8fb5b6d485ce343765d)
8E567035-2533-4A23-B7D4-F88508065355: [TV] Samsung 8 Series (65)
1931C101-AA40-4A0F-A3F2-F3A6F1946CB8: Microsoft (010920022796b587549bf212a29fe82224f4d8fa86a2db83e5c4e3)
F99C3A7D-2FD5-496E-B254-53F3314694ED: Microsoft (0109200270ed31b4ed19558a09f3fb9b323a612b89df9382022d29)
57924A3C-910A-4EAA-BFD6-3A087D3E1D57: Unknown
92A7F78A-2D6C-47C2-8808-D18901C40016: MI1S
BA4F9130-7856-4A42-AEFE-8BA9DB16136C: Samsung Electronics Co. Ltd. (4204010141ccb11a298110ceb11aac810f06000000000000)
7137C3CC-8B3F-4621-A698-118852E4705F: SENSOR_PRO
6F547DD9-AAF8-4F47-BFFE-5E1F333A8221: Microsoft (010920022796b587549bf212a29fe82224f4d8fa86a2db83e5c4e3)
67E85354-D7C6-40A3-B193-ABB2682CD5F3: Microsoft (01092002608829940f337adab443f09ce82a2d94bf560dd0a870f0)
287766F5-E065-4964-9CD3-2FAFDC10E895: Microsoft (01092002f013e0d86c43b46e97075e61411599af96d16b52